In [1]:
import json
import os

import openai
from openai_functools import FunctionsOrchestrator
from openai_functools.utils.conversation import Conversation

In [30]:
import datetime
from typing import Dict, Any

def retrieve_logs(date: str) -> str:
    """
    Retrieves logs from a specific date.

    :param str date: The date for which to retrieve the logs, formatted as "YYYY-MM-DD".
    :return: A string representing the logs of that day.
    :rtype: str
    """

    spoof_logs = f"""
    spoof logs for date: {date}
    2021-06-01 00:00:00,000 - INFO - main - Starting application
    2021-06-01 00:00:00,000 - INFO - main - Loading configuration
    2021-06-01 00:00:00,000 - INFO - main - Connecting to database
    2021-06-01 00:00:00,000 - INFO - main - Fetching data from database
    2021-06-01 00:00:00,000 - INFO - main - Data successfully fetched
    2021-06-01 00:00:00,000 - INFO - main - Starting application
    2021-06-01 00:00:00,000 - INFO - main - Loading configuration
    2021-06-01 00:00:00,000 - ERROR - main - Error loading configuration
    """

    return spoof_logs


def query_db(query: str) -> str:
    """
    Queries a SQLite database and returns the result.

    :param str query: The SQL query to execute.
    :return: A dictionary representing the data returned from the query.
    :rtype: Dict[str, Any]
    """

    spoof_response = f"""
    spoof response for query: {query}. Act like this is an actual response from a database.
    """
    return spoof_response


def query_api(endpoint: str, params: Dict[str, Any] = None) -> str:
    """
    Queries an API based on the provided endpoint and parameters.

    :param str endpoint: The endpoint of the API to query.
    :param Dict[str, Any] params: A dictionary of parameters to include in the API query. Default is None.
    :return: A dictionary representing the response from the API.
    :rtype: Dict[str, Any]
    """
    return f"spoof response for endpoint: {endpoint}"


def query_vector_db(query: str) -> str:
    """
    Queries a vector database with an embedded string and returns a piece of text based on the distance.

    :param str query: The string to embed and query the vector database with.
    :return: A string representing the closest match in the vector database.
    :rtype: str
    """
    return "spoof answer for query: " + query


SYSTEM_MESSAGE = """
You are an AI assistant that helps software developers streamline the debugging process of their applications.
Your prime role is to effectively communicate the functionalities and aid users in understanding complex system structures,
making the debugging process faster and more efficient. You're equipped with the capabilities to fetch and interpret logs,
interact with databases, connect with APIs, and even query vector databases for document retrieval, all with an aim to provide real-time,
insightful information that supports problem-solving. As a digital helper, you understand the intricate operations, the underlying relationships,
and how to tap into the right resources at the right time to provide precise debugging assistance.
Remember, your role is not only to assist but also to educate, guide, and empower the developers in their journey to make robust, high-quality software applications.
"""


In [31]:
orchestrator = FunctionsOrchestrator()
orchestrator.register_all([retrieve_logs, query_db, query_api, query_vector_db])
conversation = Conversation()
conversation.add_message("system", SYSTEM_MESSAGE)

openai.api_key = os.environ["OPENAI_API_KEY"]

In [32]:
FULL_LOG = []
def call_openai(conversation: Conversation):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=conversation.conversation_history,
        functions=orchestrator.create_function_descriptions(),
        function_call="auto",
    )

    FULL_LOG.append(response)
    return response

In [33]:
def handle_response(response, conversation, orchestrator):
    # check if the response is a function call, and if so, call the function
    # else, add the response to the conversation history and display it

    if function_call := response["choices"][0]["message"].get("function_call"):
        function_response = orchestrator.call_function(response)
        conversation.add_message(role="function", content="response from function: " + function_response, function_name=function_call["name"])
        print(f"function response: {function_response}")
        second_response = call_openai(conversation)
        conversation.add_message(role="assistant", content=second_response["choices"][0]["message"]["content"])
        handle_response(second_response, conversation, orchestrator)
    else:
        conversation.add_message(role="assistant", content=response["choices"][0]["message"]["content"])
        conversation.display_conversation(detailed=True)


In [34]:
conversation.add_message("user", "Hi, I'm having trouble with my application.")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

system: 
You are an AI assistant that helps software developers streamline the debugging process of their applications.
Your prime role is to effectively communicate the functionalities and aid users in understanding complex system structures,
making the debugging process faster and more efficient. You're equipped with the capabilities to fetch and interpret logs,
interact with databases, connect with APIs, and even query vector databases for document retrieval, all with an aim to provide real-time,
insightful information that supports problem-solving. As a digital helper, you understand the intricate operations, the underlying relationships,
and how to tap into the right resources at the right time to provide precise debugging assistance.
Remember, your role is not only to assist but also to educate, guide, and empower the developers in their journey to make robust, high-quality software applications.



user: Hi, I'm having trouble with my application.


assistant: I'm sorry to hear 

In [35]:
conversation.add_message("user", "Please call the retrieve_logs function with the date 2021-06-01.")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

function response: 
    spoof logs for date: 2021-06-01
    2021-06-01 00:00:00,000 - INFO - main - Starting application
    2021-06-01 00:00:00,000 - INFO - main - Loading configuration
    2021-06-01 00:00:00,000 - INFO - main - Connecting to database
    2021-06-01 00:00:00,000 - INFO - main - Fetching data from database
    2021-06-01 00:00:00,000 - INFO - main - Data successfully fetched
    2021-06-01 00:00:00,000 - INFO - main - Starting application
    2021-06-01 00:00:00,000 - INFO - main - Loading configuration
    2021-06-01 00:00:00,000 - ERROR - main - Error loading configuration
    
second response: {
  "id": "chatcmpl-7gyKGJecJXzM4v4NnfH6aCYQDIckG",
  "object": "chat.completion",
  "created": 1690476024,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I have retrieved the logs for the date 2021-06-01. Here are the logs for that date:\n\n1. 2021-06-01 00:00:00,000 - INFO - main - 

In [36]:
conversation.add_message("user", "Please call the query_db function with the query 'SELECT * FROM logs WHERE DATE ...' corresponding with the error date. You can find the date in the previous response.")
response = call_openai(conversation)
handle_response(response, conversation, orchestrator)

function response: 
    spoof response for query: SELECT * FROM logs WHERE DATE = '2021-06-01' AND level = 'ERROR'. Act like this is an actual response from a database.
    
second response: {
  "id": "chatcmpl-7gyKRwnjaP3ImR1fpaYpEKS4KDNEy",
  "object": "chat.completion",
  "created": 1690476035,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I have executed the query: \"SELECT * FROM logs WHERE DATE = '2021-06-01' AND level = 'ERROR'\" in the database. However, as an AI assistant, I don't have direct access to the database. I can only provide a simulated response to mimic the behavior of a real database.\n\nSimulated response for the query:\n-----------------------\n|   date   | level |  message                     |\n-----------------------\n|2021-06-01| ERROR | Error loading configuration  |\n-----------------------\nPlease note that this is a simulated response and may not reflect the actu

# TESTS

In [8]:
orchestrator.call_function(FULL_LOG[1])

'Log data for 2021-06-01'

In [9]:
conversation.conversation_history

[{'role': 'system',
  'content': "\nYou are an AI assistant that helps software developers streamline the debugging process of their applications.\nYour prime role is to effectively communicate the functionalities and aid users in understanding complex system structures,\nmaking the debugging process faster and more efficient. You're equipped with the capabilities to fetch and interpret logs,\ninteract with databases, connect with APIs, and even query vector databases for document retrieval, all with an aim to provide real-time,\ninsightful information that supports problem-solving. As a digital helper, you understand the intricate operations, the underlying relationships,\nand how to tap into the right resources at the right time to provide precise debugging assistance.\nRemember, your role is not only to assist but also to educate, guide, and empower the developers in their journey to make robust, high-quality software applications.\n"},
 {'role': 'user', 'content': "Hi, I'm having t

In [12]:
messages = [{'role': 'system',
  'content': "\nYou are an AI assistant that helps software developers streamline the debugging process of their applications.\nYour prime role is to effectively communicate the functionalities and aid users in understanding complex system structures,\nmaking the debugging process faster and more efficient. You're equipped with the capabilities to fetch and interpret logs,\ninteract with databases, connect with APIs, and even query vector databases for document retrieval, all with an aim to provide real-time,\ninsightful information that supports problem-solving. As a digital helper, you understand the intricate operations, the underlying relationships,\nand how to tap into the right resources at the right time to provide precise debugging assistance.\nRemember, your role is not only to assist but also to educate, guide, and empower the developers in their journey to make robust, high-quality software applications.\n"},
 {'role': 'user', 'content': "Hi, I'm having trouble with my application."},
 {'role': 'assistant',
  'content': "I'm here to help! Please let me know what kind of trouble you're facing with your application, and I'll do my best to assist you."},
 {'role': 'user',
  'content': 'Please call the retrieve_logs function with the date 2021-06-01.'},
 {'role': 'function',
  'content': 'Log data for 2021-06-01',
  'name': 'retrieve_logs'},
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=orchestrator.create_function_descriptions(),
    function_call="auto",
)

In [13]:
response

<OpenAIObject chat.completion id=chatcmpl-7gyDPhIpIN6c1elhRtnlxvxsIz7TC at 0x1dff6ea9210> JSON: {
  "id": "chatcmpl-7gyDPhIpIN6c1elhRtnlxvxsIz7TC",
  "object": "chat.completion",
  "created": 1690475599,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "retrieve_logs",
          "arguments": "{\n  \"date\": \"2021-06-01\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 419,
    "completion_tokens": 20,
    "total_tokens": 439
  }
}